### Phenology metric calculations

In [ ]:
!pip install geemap

In [ ]:
import ee
ee.Authenticate()
ee.Initialize(project='phenology-303215')

In [ ]:
import os
import ee
ee.Initialize()
import pandas as pd
from IPython.display import JSON
import geemap
Map = geemap.

### Functions

In [ ]:
# add dn for product aggregated into 8 days
def add_dn_date(img,beginDate=None,n=None,IncludeYear=False):
    if beginDate is None:
        beginDate = img.get('system:time_start')
    else:
        beginDate = beginDate
    if IncludeYear is False:
        IncludeYear = True
    if n is None:
        n = 8
    beginDate = ee.Date(beginDate)
    year  = beginDate.get('year')
    month = beginDate.get('month')
    diff  = beginDate.difference(ee.Date.fromYMD(year, 1, 1), 'day').add(1)
    dn    = diff.subtract(1).divide(n).floor().add(1).int()
    yearstr  = year.format('%d')
    dn = dn.format('%02d')
    return ee.Image(img).set('system:time_start', beginDate.millis()).set('date', beginDate.format('yyyy-MM-dd')).set('Year', yearstr).set('Month',beginDate.format('MM')).set('YearMonth', beginDate.format('YYYY-MM')).set('dn', dn)

# wrapper function for add_dn_date
def add_dn_date_all(Year,days):
    def wrapper(image0):
        tmp = add_dn_date(img = image0,IncludeYear=Year,n = days)
        return tmp
    return (wrapper)

# add NDVI to data
def addNDVI(image):
    #image = image.updateMask(MakMarco.eq(1))
    return image.addBands(image.normalizedDifference(['sur_refl_b02','sur_refl_b01']).rename('NDVI')).float()

# function for extracting quality bits
def getQABits(image, start, end, mascara):
    # Compute the bits we need to extract.
    pattern = 0
    for i in range(start,end+1):
        pattern += 2**i
    # Return a single band image of the extracted QA bits, giving the     band a new name.
    return image.select([0], [mascara]).bitwiseAnd(pattern).rightShift(start)

# mask out low quality pixels (based on flags)
def maskPixels(image0):
    #Select the QA band
    QA = image0.select('StateQA')
    # Get the land_water_flag bits
    landWaterFlag = getQABits(QA, 3, 5, 'land_water_flag')
    #Get the cloud_state bits and find cloudy areas.
    cloud = getQABits(QA, 0, 1, 'cloud_state').expression("b(0) == 1 || b(0) == 2")
    # Get the cloud_shadow bit
    cloudShadows = getQABits(QA, 2, 2, 'cloud_shadow')
    # Get the Pixel is adjacent to cloud bit
    cloudAdjacent = getQABits(QA, 13, 13, 'cloud_adj')
    # Get the internal cloud flag
    cloud2 = getQABits(QA, 10, 10, 'cloud_internal')
    # Get the internal fire flag
    fire = getQABits(QA, 11, 11, 'fire_internal')
    # Get the MOD35 snow/ice flag
    snow1 = getQABits(QA, 12, 12, 'snow_MOD35')
    # Get the internal snow flag
    snow2 = getQABits(QA, 15, 15, 'snow_internal')
    # create mask
    mask = landWaterFlag.eq(1).And(cloud.Not()).And(cloudShadows.Not()).And(cloudAdjacent.Not()).And(cloud2.Not()).And(fire.Not()).And(snow1.Not()).And(snow2.Not())
    return image0.updateMask(mask)

# combine two images (using match image)
def imgstobands(image):
    return image.addBands(image.get('match'))

# identify images which should not be used
def smoothed_redmin(image):
    tmp = image.reproject(crs = 'SR-ORG:6974',scale=463.3127165275).\
        reduceResolution(ee.Reducer.min(),bestEffort = False, maxPixels = 65536).\
        reproject(crs = ee.Projection('EPSG:4326').scale(0.05 , 0.05)).updateMask(1)
    return tmp.select('NDVI').rename('NDVI_5km')


# count pixels
def smoothed_redcount1(image):
    tmp = image.reproject(crs = 'SR-ORG:6974',scale=463.3127165275).\
        reduceResolution(ee.Reducer.count(),bestEffort = False, maxPixels = 65536).\
        reproject(crs = ee.Projection('EPSG:4326').scale(0.05 , 0.05)).updateMask(1)
    return tmp.select('NDVI').rename('NDVI_5km_X')


# count pixels
def smoothed_redcount2(image):
    tmp = image.reproject(crs = 'SR-ORG:6974',scale=463.3127165275).\
        reduceResolution(ee.Reducer.count(),bestEffort = False, maxPixels = 65536).\
        reproject(crs = ee.Projection('EPSG:4326').scale(0.05 , 0.05)).updateMask(1)
    return tmp.select('NDVI').rename('NDVI_5km_Y')

# filter smoothed data: fill in images with 0s. This gives all the pixels theoretically available
def filt_smoothed_c(image):
    image = image.select('NDVI')
    image = image.unmask()
    image = image.where(image.eq(0),0)
    return image.updateMask(count_valid.gte(15)).updateMask(forestmask.eq(1))

# filter smoothed data: this keeps usable pixels
def filt_smoothed2_c(image):
    image = image.select('NDVI')
    #image = image.unmask()
    #image = image.where(image.eq(0),-9999)
    return image.updateMask(count_valid.gte(15)).updateMask(forestmask.eq(1))

# calculate delta
def deltac(image):
    return image.select('NDVI_5km_X').subtract(image.select('NDVI_5km_Y')).rename('delta')

# filter images by delta
def delta_filt(image):
    return image.select('NDVI').updateMask(ee.Image(image.get('match')).lt(10))


def smooth_func(image):
    collection = ee.ImageCollection.fromImages(image.get('images'))
    return ee.Image(image).addBands(collection.mean().rename(['mean']))

#def clim5y(month):
#    month = ee.String(month)
#    seqNDVI = MOD09ndviY.filterMetadata('dn', 'equals',month)
#    return seqNDVI.median().copyProperties(seqNDVI.first(), ['system:time_start','system:time_end','dn','Month'])


def clim5y(month):
    seqNDVI1 = MOD09ndviY1.filterMetadata('dn', 'equals',month).first()
    # select image from year of interest
    seqNDVI2 = MOD09ndviY2.filterMetadata('dn', 'equals',month)
    inversemask = seqNDVI1.gt(-1).selfMask()
    median = seqNDVI2.map(lambda image: image.updateMask(inversemask.unmask().Not())).median()
    final = ee.ImageCollection.fromImages([seqNDVI1,median]).median()
    return final.copyProperties(seqNDVI1, ['system:time_start','system:time_end','dn','Month'])

def clim5y1(month):
    seqNDVI1 = MOD09ndviY1.filterMetadata('dn', 'equals',month).first() # focal year
    seqNDVI2 = MOD09ndviY2.filterMetadata('dn', 'equals',month).median() # five years ahead
    # combine together
    nodata = -9999
    mask = seqNDVI1.mask()
    img = seqNDVI1.unmask(nodata)
    img = img.where(mask.Not(), seqNDVI2)
    img = img.updateMask(img.neq(nodata))
    return img #img.copyProperties(seqNDVI1, ['system:time_start','system:time_end','dn','Month'])



def tempfilt(image):
    mask = ee.Image(ee.List(image.get('match')).get(0))
    return image.updateMask(mask)

# function for accumulating NDVI
def accumulate(image,list):
    # Get the latest cumulative NDVI of the list with
    # get(-1).  Since the type of the list argument to the function is unknown,
    # it needs to be cast to a List.  Since the return type of get() is unknown,
    # cast it to Image.
    previous = ee.Image(ee.List(list).get(-1)).toFloat().unmask()
    # Add the current anomaly to make a new cumulative NDVI image and Propagate metadata to the new image.
    added = image.unmask().toFloat().add(previous).toFloat().set('system:time_start', image.get('system:time_start'))
    return ee.List(list).add(added)

# cumulative normalized
def cum_dividelast(image):
    tmp = image.divide(last).multiply(Cumulative_mean_at_5km)
    return tmp

# calculate deviations
def deviations_calc(image):
    tmp = image.select('NDVI').setDefaultProjection(crs = 'SR-ORG:6974',scale = 463.3127165275).reduceResolution(ee.Reducer.stdDev(), bestEffort = False, maxPixels = 150).reproject(ee.Projection('EPSG:4326').scale(0.05, 0.05)).updateMask(1).rename('stdDev')
    return tmp


def addTimeBand(img):
    ## make sure mask is consistent ##
    mask = img.mask()
    time = img.metadata('system:time_start').rename("time").mask(mask)
    return img.addBands(time)


def replace_mask(img, newimg, nodata):
    if frame is None:
        nodata = 8
    # var con = img.mask();
    # var res = img., NODATA
    mask = img.mask()
    # The only nsolution is unmask & updatemask */
    img = img.unmask(nodata)
    img = img.where(mask.Not(), newimg)
    img = img.updateMask(img.neq(nodata))
    return img

def linearInterp(imgcol,frame = None,nodata = None):
    if frame is None:
        frame = 32
    if nodata is None:
        nodata = 0
    timestart   = 'system:time_start'
    imgcol = imgcol.map(addTimeBand)

    # We'll look for all images up to 32 days away from the current image (if frame not provided!)
    maxDiff = ee.Filter.maxDifference(frame * (1000*60*60*24), timestart, None, timestart)

    #cond    = {'leftField':timestart, 'rightField':timestart}
    # Images after, sorted in descending order (so closest is last).
    #var f1 = maxDiff.and(ee.Filter.lessThanOrEquals(time, null, time))
    f1 = ee.Filter.And(maxDiff, ee.Filter.lessThanOrEquals(leftField = timestart,rightField = timestart))
    c1 = ee.Join.saveAll(matchesKey = 'after', ordering = timestart, ascending = False).apply(imgcol, imgcol, f1)
    # Images before, sorted in ascending order (so closest is last).
    # var f2 = maxDiff.and(ee.Filter.greaterThanOrEquals(time, null, time))

    f2 = ee.Filter.And(maxDiff, ee.Filter.greaterThanOrEquals(leftField = timestart,rightField = timestart))
    c2 = ee.Join.saveAll(matchesKey = 'before', ordering = timestart, ascending = True).apply(c1, imgcol, f2)

 # interpolation
    def func_its(img):
        img = ee.Image(img)
        before = ee.ImageCollection.fromImages(ee.List(img.get('before'))).mosaic()
        after  = ee.ImageCollection.fromImages(ee.List(img.get('after'))).mosaic()
        img = img.set('before', {}).set('after', {})

        # constrain after or before no NA values, confirm linear Interp having result
        before = replace_mask(before, after, nodata)
        after  = replace_mask(after , before, nodata)

        # Compute the ratio between the image times.
        x1 = before.select('time').double()
        x2 = after.select('time').double()
        now = ee.Image.constant(img.date().millis()).double()
        ratio = now.subtract(x1).divide(x2.subtract(x1))  # this is zero anywhere x1 = x2

        # Compute the interpolated image.
        before = before.select(0); #remove time band now
        after  = after.select(0)
        img    = img.select(0)
        interp = after.subtract(before).multiply(ratio).add(before)
        qc = img.mask().Not().rename('qc')
        interp = replace_mask(img, interp, nodata)

        # Map.addLayer(interp, {}, 'interp')
        return interp.addBands(qc).copyProperties(img, img.propertyNames())
    interpolated = ee.ImageCollection(c2.map(func_its))
    return interpolated


### Load in collections and required images

In [ ]:
# load collections and required images
collection = ee.ImageCollection('MODIS/006/MOD09A1').filterDate('2000-01-01', '2020-12-31')
forestmask = ee.Image('users/marcogirardello/phenoutils/mask_unchanged_500m')
smallgrid = ee.FeatureCollection('users/marcogirardello/phenoutils/grid_export_phenology22')
terracl = ee.Image("users/marcogirardello/phenoutils/terraclimate_months")

#### Create monthly binary masks based on TERRACLIMATE long-term averages

In [ ]:
# convert into image collection
terracl1 = ee.ImageCollection.fromImages(terracl.bandNames().map(lambda name: terracl.select([name])))
# add month property taken from bands
terracl2 = terracl1.map(lambda image: image.set(ee.Dictionary({'Month': ee.String(image.bandNames().get(0)).slice(2)})))
# rename bands into b1
terracl3 = terracl2.map(lambda image: image.rename('avgtemp'))
# create binary map showing where temperature is greater than 0
terraclmonths = terracl3.map(lambda image: image.gt(0).copyProperties(image,['Month']))
terraclimbin = terraclmonths.map(lambda image: image.where(image.gt(0),1))
# sum of pixel count across 12 months where temperature > 0 °C
ImgCountDistinct = terraclimbin.reduce(ee.Reducer.sum())
# create pixel specific filter: i.e. number of observations that one needs to have in the time series for cumulating
NumberOfMinImagePerYear = ImgCountDistinct.multiply(2)

### <span style="color:blue">Pre-processing step 1: filtering data by quality flags and calculated NDVI.</span>
This include snow, cloud, fire, cloud shadows and the land/water mask

In [ ]:
# end and start date of period of interest
start_date = ee.Date.fromYMD(2001, 1, 1)
end_date   = ee.Date.fromYMD(2020, 12, 31)

In [ ]:
# add period (from 1 to 46 every year)
collection = collection.map(add_dn_date_all(Year = False, days = 8))

In [ ]:
# mask out low quality pixels
MOD09masked = collection.filterDate(start_date, end_date).map(maskPixels)

In [ ]:
# add NDVI as a new band
MOD09ndvi = MOD09masked.map(addNDVI).select('NDVI')

### <span style="color:blue">Main calculations</span>

In [ ]:
# Generate a list of period identifiers (01 to 46) for MODIS data aggregation, and convert to Earth Engine List object.
tmpseas = ["%02d" % x for x in list(range(1, 46+1))]
tmpseas1 = ee.List(tmpseas)

In [ ]:
# Focal year (should be change according to the year one wishes to export)
# This will ensure that only one year at the time is processed
year = 2003

In [ ]:
# temporal window used for interpolation
frame  = 8*8
nodata = -9999

# filter year range only for focal year
start_date = ee.Date.fromYMD(year,1,1)
end_date = ee.Date.fromYMD(year,12,31)
MOD09ndviY1_tmp = MOD09ndvi.filterDate(start_date, end_date)

# linear interpolation for focal years
MOD09ndviY1 = linearInterp(MOD09ndviY1_tmp, frame, nodata).select('NDVI')

if year > 2015:
    # temporary year set to 2015
    yeartmp = 2015
    # filter for additional four years
    MOD09ndviY2 = MOD09ndvi.filterDate(ee.Date.fromYMD(yeartmp+1,1,1),ee.Date.fromYMD(yeartmp+5,12,31))
else:
    # filter for additional four years
    MOD09ndviY2 = MOD09ndvi.filterDate(ee.Date.fromYMD(year+1,1,1),ee.Date.fromYMD(year+5,12,31))

# climatology 5 years (monthly composites)
seasons2 = ee.ImageCollection.fromImages(tmpseas1.map(clim5y1))

# use climatology to filter collection

# join with terraclimate data
jfilter = ee.Filter.equals(leftField = 'Month',rightField = 'Month')
simplej = ee.Join.inner()
seasons2j = simplej.apply(seasons2,terraclmonths,jfilter)

# add temperature mask as a new band
seasons2 = ee.ImageCollection(seasons2j.map(lambda joinresult: ee.Image(joinresult.get('primary')).addBands(joinresult.get('secondary'))))

# filter for mean annual temperatures > 0 °C
seasons2 = seasons2.map(lambda image: image.updateMask(image.select('avgtemp').gt(0)))

# number of valid images
count_valid = seasons2.select('NDVI').count()

# make sure that a minimum number
smoothed  = seasons2.map(lambda image: image.updateMask(count_valid.gte(15)).updateMask(forestmask.eq(1)))

# get image count
ct = smoothed.select('NDVI').count()

# maximum number of pixels available within a pixel of 5km x 5km
ctmax  = ct.setDefaultProjection(crs = 'SR-ORG:6974',scale = 463.3127165275).\
                          reduceResolution(ee.Reducer.max(), bestEffort = False, maxPixels = 150).\
                          reproject(ee.Projection('EPSG:4326').scale(0.05, 0.05)).updateMask(1)

# mean number of pixels available within a pixel of 5km x 5km
ctmean  = ct.setDefaultProjection(crs = 'SR-ORG:6974',scale = 463.3127165275).\
                          reduceResolution(ee.Reducer.mean(), bestEffort = False, maxPixels = 150).\
                          reproject(ee.Projection('EPSG:4326').scale(0.05, 0.05)).updateMask(1)

# get the timestamp from the most recent image in the reference collection.
time0 = smoothed.first().get('system:time_start')

# rename the first band 'NDVI'.
first = ee.List([ee.Image(0).set('system:time_start', time0).\
                 select([0],['NDVI']).toFloat().\
                 set(ee.Dictionary({'Month': '01'}))])

# recast result into a list object
cumulative = ee.ImageCollection(ee.List(smoothed.iterate(accumulate, first)))

# filter out images where NDVI > 0
cumulative = cumulative.map(lambda image: image.select('NDVI').updateMask(image.select('NDVI').gt(0)))

# sort collection for normalisation
last = cumulative.sort('system:time_start', False).first()
last = last.updateMask(last.gte(3.5))

# mean of cumulative values
Cumulative_mean = last.divide(46)
Cumulative_mean_at_5km = Cumulative_mean.setDefaultProjection(crs = 'SR-ORG:6974',
                                                              scale = 463.3127165275).\
                                                              reduceResolution(ee.Reducer.mean(),
                                                              bestEffort = False, maxPixels = 150).\
                                                              reproject(ee.Projection('EPSG:4326').\
                                                                        scale(0.05, 0.05)).\
                                                                        updateMask(1)

# cumulative map normalised
cumulativeNorm = cumulative.map(cum_dividelast)

# deviations from mean phenological trajectory
cumulativeStd10 = cumulativeNorm.map(deviations_calc)

# get completness of observational coverage
pixelmask = ctmean.eq(ctmax)
prop = ctmean.divide(ctmax)

# calculate the mean of the phenological deviations, rescale and filter completeness
cumulativeStd10_mean = cumulativeStd10.mean().multiply(10000).updateMask(prop.gte(0.98))